# Part X — Advanced Topics  
## 45. Advanced Admin and Internal Tools (Dashboards, Custom Admin Views, Performance, Safety)

Django Admin can be more than CRUD screens. In many real organizations, Django
Admin becomes:

- internal dashboards
- operational tools (retries, reprocessing jobs)
- content moderation consoles
- reporting tools
- support tools (impersonation carefully, user lookup)
- data quality tools (integrity checks, cleanup)

This chapter teaches “advanced admin” in an industry-standard, safe way:

- custom admin pages and dashboards
- admin performance optimization
- safe internal actions and reprocessing
- bulk operations with confirmation
- admin-only reporting with CSV export
- permission discipline (not everything is superuser-only, but not everything is open)
- audit logging for internal actions

We’ll implement:
- an “Operations Dashboard” in admin showing:
  - failed export jobs
  - failed webhook events
  - recently published articles
- an admin action to “retry failed export job”
- an admin action to “reprocess webhook event”
- query optimizations so admin remains fast

---

## 45.0 Learning Outcomes

By the end, you should be able to:

1. Customize admin UI at a professional level:
   - custom list displays, filters, querysets
   - custom admin views (non-model pages)
2. Build internal dashboards using admin templates.
3. Add safe admin actions that:
   - enforce permissions
   - are idempotent
   - log/audit the action
4. Optimize admin performance (avoid N+1; annotate/prefetch; index awareness).
5. Provide internal reporting/export tools safely.
6. Write tests for critical admin custom actions and permissions (smoke tests).

---

## 45.1 Admin as an Internal Product (Mindset)

Treat admin tools like product features:
- correctness matters
- security matters
- UX matters (staff efficiency)
- performance matters at scale

Admin tools often have higher “blast radius” than user-facing features because they
can mass edit and reprocess events.

---

## 45.2 Admin Performance Optimization (Repeatable Patterns)

### 45.2.1 Never do per-row DB queries in list_display
If list_display references related fields or computed values:
- use `select_related` for FK/O2O
- use `prefetch_related` for M2M
- use annotation for counts

You learned these earlier. In admin they are essential.

### 45.2.2 Use `get_queryset` in ModelAdmin
Example (export jobs list):

```python
def get_queryset(self, request):
    qs = super().get_queryset(request)
    return qs.select_related("organization", "created_by")
```

### 45.2.3 Use database indexes for filters/order
If you filter by `status`, `created_at`, make sure you have indexes (you added some).
Admin is essentially a reporting UI; indexes matter.

---

## 45.3 Build a Real Ops Dashboard (Custom Admin View)

We’ll add a custom page accessible from admin:
- `/admin/ops/` showing system operational info

### 45.3.1 Create an admin “site” view (simple pattern)
We can implement a custom admin view by adding a URL under admin.

Create `config/admin_ops.py`:

```python
from __future__ import annotations

from django.contrib import admin
from django.contrib.admin.views.decorators import staff_member_required
from django.template.response import TemplateResponse
from django.utils import timezone

from articles.models import Article
from integrations.models import WebhookEvent
from tasks.models import TaskExportJob


@staff_member_required
def ops_dashboard(request):
    now = timezone.now()

    failed_exports = (
        TaskExportJob.objects.filter(status=TaskExportJob.Status.FAILED)
        .select_related("organization", "created_by")
        .order_by("-created_at")[:20]
    )

    pending_exports = (
        TaskExportJob.objects.filter(status__in=[TaskExportJob.Status.PENDING, TaskExportJob.Status.RUNNING])
        .select_related("organization", "created_by")
        .order_by("-created_at")[:20]
    )

    failed_webhooks = (
        WebhookEvent.objects.filter(status=WebhookEvent.Status.FAILED)
        .order_by("-received_at")[:20]
    )

    recent_published = (
        Article.objects.filter(status=Article.Status.PUBLISHED)
        .select_related("author")
        .order_by("-published_at")[:10]
    )

    context = {
        **admin.site.each_context(request),
        "title": "Operations Dashboard",
        "failed_exports": failed_exports,
        "pending_exports": pending_exports,
        "failed_webhooks": failed_webhooks,
        "recent_published": recent_published,
        "now": now,
    }

    return TemplateResponse(request, "admin/ops_dashboard.html", context)
```

### 45.3.2 Wire the URL into admin
Edit `config/urls.py` and extend admin URLs by overriding `admin.site.get_urls` is
the classic approach. But simplest is to add a path under admin via URLconf:

```python
from django.contrib import admin
from django.urls import path, include

from config.admin_ops import ops_dashboard

urlpatterns = [
    path("admin/ops/", ops_dashboard, name="ops_dashboard"),
    path("admin/", admin.site.urls),
    # ...
]
```

#### Why this works
- `/admin/ops/` is protected by `staff_member_required`
- It is still part of the same domain and admin context
- You can link it from admin templates

### 45.3.3 Create the admin dashboard template
Create `templates/admin/ops_dashboard.html`:

```django
{% extends "admin/base_site.html" %}

{% block content %}
  <h1>{{ title }}</h1>

  <p>Now: {{ now }}</p>

  <h2>Pending/Running Exports</h2>
  <table class="admin-table">
    <thead>
      <tr>
        <th>Job</th>
        <th>Org</th>
        <th>Created by</th>
        <th>Status</th>
        <th>Created</th>
      </tr>
    </thead>
    <tbody>
      {% for job in pending_exports %}
        <tr>
          <td>
            <a href="{% url 'admin:tasks_taskexportjob_change' job.id %}">#{{ job.id }}</a>
          </td>
          <td>{{ job.organization.slug }}</td>
          <td>{{ job.created_by.username }}</td>
          <td>{{ job.status }}</td>
          <td>{{ job.created_at }}</td>
        </tr>
      {% empty %}
        <tr><td colspan="5"><em>None</em></td></tr>
      {% endfor %}
    </tbody>
  </table>

  <h2>Failed Exports</h2>
  <table class="admin-table">
    <thead>
      <tr>
        <th>Job</th>
        <th>Org</th>
        <th>Status</th>
        <th>Error</th>
      </tr>
    </thead>
    <tbody>
      {% for job in failed_exports %}
        <tr>
          <td><a href="{% url 'admin:tasks_taskexportjob_change' job.id %}">#{{ job.id }}</a></td>
          <td>{{ job.organization.slug }}</td>
          <td>{{ job.status }}</td>
          <td><code>{{ job.error|truncatechars:120 }}</code></td>
        </tr>
      {% empty %}
        <tr><td colspan="4"><em>None</em></td></tr>
      {% endfor %}
    </tbody>
  </table>

  <h2>Failed Webhook Events</h2>
  <table class="admin-table">
    <thead>
      <tr>
        <th>Provider</th>
        <th>Event ID</th>
        <th>Status</th>
        <th>Received</th>
      </tr>
    </thead>
    <tbody>
      {% for e in failed_webhooks %}
        <tr>
          <td>{{ e.provider }}</td>
          <td><a href="{% url 'admin:integrations_webhookevent_change' e.id %}">{{ e.event_id }}</a></td>
          <td>{{ e.status }}</td>
          <td>{{ e.received_at }}</td>
        </tr>
      {% empty %}
        <tr><td colspan="4"><em>None</em></td></tr>
      {% endfor %}
    </tbody>
  </table>

  <h2>Recently Published Articles</h2>
  <ul>
    {% for a in recent_published %}
      <li>
        <a href="{% url 'admin:articles_article_change' a.id %}">{{ a.title }}</a>
        by {{ a.author.username }}
      </li>
    {% empty %}
      <li><em>None</em></li>
    {% endfor %}
  </ul>
{% endblock %}
```

### 45.3.4 Add a link to Ops Dashboard in admin index (optional)
You can override `admin/index.html`, but simplest is to add it to your own docs or
nav. Many teams create a “staff portal” instead of hacking admin templates.

If you want it in admin index, you can override `templates/admin/index.html` and
add a link.

---

## 45.4 Admin Actions: “Retry Failed Export Job” (Safe + Audited)

We’ll add an admin action on `TaskExportJob`:

- Only allow retry when status is FAILED
- Reset status to PENDING
- Enqueue Celery task to run again
- Write an audit log entry

### 45.4.1 Register TaskExportJob in admin (if not already)
In `tasks/admin.py`:

```python
from django.contrib import admin
from django.utils import timezone

from audit.services import log_event
from tasks.models import TaskExportJob
from tasks.tasks_exports import run_task_export_job_task


@admin.register(TaskExportJob)
class TaskExportJobAdmin(admin.ModelAdmin):
    list_display = ("id", "organization", "created_by", "status", "created_at", "finished_at")
    list_filter = ("status", "organization", "created_at")
    search_fields = ("organization__slug", "created_by__username", "id")
    ordering = ("-created_at",)
    actions = ["retry_failed_exports"]

    def get_queryset(self, request):
        qs = super().get_queryset(request)
        return qs.select_related("organization", "created_by")

    @admin.action(description="Retry failed export jobs")
    def retry_failed_exports(self, request, queryset):
        # Only allow staff with change permission.
        if not self.has_change_permission(request):
            self.message_user(request, "No permission.", level="error")
            return

        failed = queryset.filter(status=TaskExportJob.Status.FAILED)
        count = 0

        for job in failed:
            job.status = TaskExportJob.Status.PENDING
            job.error = ""
            job.started_at = None
            job.finished_at = None
            job.save(update_fields=["status", "error", "started_at", "finished_at"])

            run_task_export_job_task.delay(job.id)
            count += 1

            try:
                log_event(
                    actor=request.user,
                    action="task_export.retry",
                    target=job,
                    details={"job_id": job.id},
                )
            except Exception:
                pass

        self.message_user(request, f"Retried {count} failed export job(s).")
```

### Why we iterate instead of bulk update
We want to:
- enqueue a task per job
- audit each retry action
A bulk update would lose those per-job side effects.

### Safety note
- Admin actions can run on many rows. Consider limiting or adding a confirmation
  step if you expect large sets.

---

## 45.5 Admin Actions: “Reprocess Webhook Event” (Idempotent)

Reprocessing webhooks is extremely useful for ops:
- provider had temporary outage
- your worker failed
- you fixed a bug and want to replay

### 45.5.1 Register WebhookEvent in admin with reprocess action
In `integrations/admin.py`:

```python
from django.contrib import admin
from django.utils import timezone

from audit.services import log_event
from integrations.models import WebhookEvent
from integrations.webhooks.tasks import process_webhook_event_task


@admin.register(WebhookEvent)
class WebhookEventAdmin(admin.ModelAdmin):
    list_display = ("id", "provider", "event_id", "status", "received_at", "processed_at")
    list_filter = ("provider", "status", "received_at")
    search_fields = ("event_id",)
    ordering = ("-received_at",)
    actions = ["reprocess_events"]

    @admin.action(description="Reprocess selected webhook events")
    def reprocess_events(self, request, queryset):
        if not self.has_change_permission(request):
            self.message_user(request, "No permission.", level="error")
            return

        count = 0
        for e in queryset:
            # Reset status only if you want; sometimes you want to preserve status and just retry processing.
            e.status = WebhookEvent.Status.RECEIVED
            e.error = ""
            e.processed_at = None
            e.save(update_fields=["status", "error", "processed_at"])

            process_webhook_event_task.delay(e.id)
            count += 1

            try:
                log_event(
                    actor=request.user,
                    action="webhook.reprocess",
                    target=e,
                    details={"provider": e.provider, "event_id": e.event_id},
                )
            except Exception:
                pass

        self.message_user(request, f"Enqueued reprocessing for {count} event(s).")
```

### Why “reprocessing” must be idempotent
If the webhook handler creates TaskEvents or updates statuses, replaying can cause
duplicates unless you built idempotency keys. A professional replay system requires:
- dedupe markers per event (unique constraint)
- or update-only semantics

Your `WebhookEvent` unique constraint prevents duplicate receipt, not duplicate
processing side effects. Your processing logic must handle replay safely.

---

## 45.6 Custom Admin Filters and Queryset Scoping (Optional)

For internal staff tools, you might filter by:
- organization slug
- actor username
- date range
- status

Django admin filters are powerful, but performance depends on indexes.

---

## 45.7 Add a Safe Internal “CSV Export” Admin View (Optional)

Sometimes you want:
- a “download report” button in admin
- a custom query that uses existing filters
- limited to staff

Design principles:
- enforce permissions
- log export actions (audit)
- use streaming for large results

You already built export in the app; admin exports are a variation of that.

---

## 45.8 Admin Security (Advanced Realities)

### 45.8.1 Admin is a high-value target
- brute force login attempts
- privilege escalation
- CSRF/XSS impact is severe in admin

Minimum:
- strong passwords + MFA/SSO (if possible)
- restrict admin to staff only (default)
- consider IP allowlist/VPN for internal admin

### 45.8.2 Never trust admin actions to be “safe”
Even staff make mistakes. Add:
- confirmations for destructive actions
- reversible actions where possible
- audit logs

---

## 45.9 Testing Advanced Admin (High-Value Tests Only)

You generally test:
- admin pages load (smoke)
- custom actions do the right DB changes and enqueue tasks (mock)
- permission restrictions

### 45.9.1 Example: test retry export action triggers Celery task
In tests, run Celery eager or mock `.delay`.

Simple approach: patch the delay function.

```python
from unittest.mock import patch

from django.contrib.auth import get_user_model
from django.test import TestCase
from django.urls import reverse

from orgs.models import Organization
from tasks.models import TaskExportJob


class AdminRetryExportTests(TestCase):
    def setUp(self):
        User = get_user_model()
        self.admin = User.objects.create_superuser(
            username="admin",
            email="admin@example.com",
            password="pass12345",
        )
        self.client.force_login(self.admin)

        org = Organization.objects.create(name="Acme", slug="acme")
        self.job = TaskExportJob.objects.create(
            organization=org,
            created_by=self.admin,
            filters={},
            status=TaskExportJob.Status.FAILED,
            error="boom",
        )

    @patch("tasks.admin.run_task_export_job_task.delay")
    def test_retry_action_enqueues(self, delay_mock):
        url = reverse("admin:tasks_taskexportjob_changelist")

        response = self.client.post(
            url,
            {
                "action": "retry_failed_exports",
                "_selected_action": [self.job.id],
            },
            follow=True,
        )
        self.assertEqual(response.status_code, 200)

        self.job.refresh_from_db()
        self.assertEqual(self.job.status, TaskExportJob.Status.PENDING)
        delay_mock.assert_called_once_with(self.job.id)
```

This is a high-signal admin test:
- ensures action name works
- ensures state changes happen
- ensures background job is enqueued

---

## 45.10 Exercises (Do These Before Proceeding)

1. Implement the Ops Dashboard page at `/admin/ops/` and ensure only staff can see it.
2. Add “Retry export job” admin action and test it (mocking Celery delay).
3. Add “Reprocess webhook event” admin action and test it.
4. Add admin performance improvements:
   - `select_related` for relevant list views
   - annotate counts where needed
5. Add audit logs for:
   - export retry
   - webhook reprocess
   - comment approve / delete actions

---

## 45.11 Chapter Summary

- Django admin can become powerful internal tooling when extended intentionally.
- Custom admin views (dashboards) can centralize ops visibility.
- Admin actions must be safe:
  - permission-checked
  - idempotent
  - auditable
- Admin performance must be optimized like any other UI (avoid N+1, use indexes).
- Test only critical admin custom logic, not every UI detail.

---

Next chapter: **46. Advanced Testing and Quality Engineering**  
We’ll go beyond basic tests into property-based testing, contract testing for APIs,
load testing, and building quality gates (coverage, performance budgets) that match
industry expectations for mature Django systems.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='44. accessibility_and_ux_quality.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='46. advanced_testing_and_quality_engineering.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
